### TODO

Core Features
- Handle missing values in structured outputs
- Test other APIs - use env variables in yaml
- Consider shipping JUST the sculptors as a library
- Why is it making up fields when I use Deepinfra?

Helper Improvements - maybe ship separately
- Test helpers for datasources
- String together steps
- Clean up visualizer
- Delete extra files

In [3]:
%load_ext autoreload
%autoreload 2

import os
import toml

secrets = toml.load("secrets.toml")
os.environ["OPENAI_API_KEY"] = secrets["openai"]["api_key"]
os.environ["DEEPINFRA_API_KEY"] = secrets["deepinfra"]["api_key"]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from sculptor import Sculptor
from sculptor import SculptorPipeline
pipeline = SculptorPipeline.from_config('pipeline_config.yaml')
results = pipeline.process(your_data)

In [2]:
import pandas as pd
from sculptor import Sculptor
from sculptor import SculptorPipeline
from helpers.data_sources import CSVDataSource

# 1. Create our sculptors
relevance_sculptor = Sculptor(
    schema={
        "is_valid_sample": {"type": bool, "description": "True only if this text contains demographic information."},
        "explanation": {"type": str, "description": "Explain why this sample is or is not valid."}
    },
    instructions="Determine if the following text contains demographic information about a person.",
    template="Text: {text}"
)

demo_sculptor = Sculptor.from_config("sample_data/demosculpt.yaml")
demo_sculptor.add("first_letter", str, "First letter of the persons first name")

# 2. Create and configure the pipeline
pipeline = (SculptorPipeline()
    .add(relevance_sculptor, lambda x: x['is_valid_sample'])  # Filter on is_valid_sample
    .add(demo_sculptor))

# 3. Load and process the data
csv_source = CSVDataSource("sample_data/people.csv")
df = csv_source.get_data()

# 4. Run the pipeline (will preserve all columns by default)
results = pipeline.process(
    df.to_dict('records'),
    n_workers=4,  # Parallel processing
    show_progress=True
)

# 5. Convert back to DataFrame
extracted_df = pd.DataFrame(results)

# Display results
extracted_df.head(20)


Step 1/2


Processing items: 100%|██████████| 11/11 [00:03<00:00,  3.66it/s]



Step 2/2


Processing items: 100%|██████████| 8/8 [00:02<00:00,  3.25it/s]


,text,is_valid_sample,explanation,name,age,city,occupation,interests,is_married,num_children,net_worth,first_letter
0,"Alice is 30 years old, lives in New York, and ...",True,The text contains demographic information such...,Alice,30,New York,software engineer,"[hiking, reading]",False,1,1200000,A
1,"Bob, 25, is a teacher in London. He's an avid ...",True,The text contains demographic information such...,Bob,25,London,teacher,[cycling],True,2,500000,B
2,Charlie is a 40-year-old data scientist from C...,True,The text contains demographic information such...,Charlie,40,Chicago,data scientist,"[skiing, cooking, photography]",True,1,800000,C
3,"David, a 35-year-old architect, resides in San...",True,The text contains demographic information such...,David,35,San Francisco,architect,[rock climbing],False,0,1500000,D
4,Emily is a 28-year-old nurse in Seattle. She l...,True,The text contains demographic information such...,Emily,28,Seattle,nurse,"[traveling, trying new foods]",False,0,400000,E
5,Frank is a 50-year-old lawyer living in Boston...,True,The text contains demographic information such...,Frank,50,Boston,lawyer,"[golfing, fishing]",True,3,3200000,F
6,"Grace, a 22-year-old student in Austin, is pas...",True,The text contains demographic information such...,Grace,22,Austin,student,"[music, volunteering]",False,0,0,G
7,"Katrina, a 28-year-old art expert in NYC.",True,The text contains demographic information such...,Katrina,28,NYC,art expert,[],False,0,0,K


In [31]:
import pandas as pd
from sculptor import Sculptor
from helpers.data_sources import CSVDataSource

# 1. Create a Sculptor for relevance checking - using extraction_schema
relevance_sculptor = Sculptor(
    schema={"is_valid_sample": {"type": bool, "description": "True only if this text contains demographic information.",},
            "explanation": {"type": str,"description": "Explain why this sample is or is not valid.",},},
    instructions="Determine if the following text contains demographic information about a person.",
    template="Text: {text}",
)

# 2. Create a Sculptor for data extraction - using add()
demo_sculptor = Sculptor.from_config("sample_data/demosculpt.yaml")
demo_sculptor.add("first_letter", str, "First letter of the persons first name")  # You can add fields like this

csv_source = CSVDataSource("sample_data/people.csv")
df = csv_source.get_data()

# 3. Apply the relevance sculptor
df["relevance_data"] = df.apply(lambda row: relevance_sculptor.sculpt(row), axis=1)
df["is_valid_sample"] = df["relevance_data"].apply(lambda x: x.get("is_valid_sample"))
df["explanation"] = df["relevance_data"].apply(lambda x: x.get("explanation"))
df.drop(columns=["relevance_data"], inplace=True)  # Drop the intermediate column

# 4. Filter based on relevance
relevant_df = df[df["is_valid_sample"] == True].copy()

# 5. Apply the data extraction sculptor
relevant_df["extracted_data"] = relevant_df.apply(lambda row: demo_sculptor.sculpt(row), axis=1)

# 6. Expand the extracted data into separate columns
extracted_df = pd.concat([relevant_df, relevant_df["extracted_data"].apply(pd.Series)], axis=1)
extracted_df.drop(columns=["extracted_data", "is_valid_sample", "explanation"], inplace=True)

extracted_df.head(20)

,text,name,age,city,occupation,interests,is_married,num_children,net_worth,first_letter
0,"Alice is 30 years old, lives in New York, and ...",Alice,30,New York,software engineer,"[hiking, reading]",False,1,1200000,A
1,"Bob, 25, is a teacher in London. He's an avid ...",Bob,25,London,teacher,[cycling],True,2,500000,B
3,Charlie is a 40-year-old data scientist from C...,Charlie,40,Chicago,data scientist,"[skiing, cooking, photography]",True,1,800000,C
5,"David, a 35-year-old architect, resides in San...",David,35,San Francisco,architect,[rock climbing],False,0,1500000,D
6,Emily is a 28-year-old nurse in Seattle. She l...,Emily,28,Seattle,nurse,"[traveling, trying new foods]",False,0,400000,E
8,Frank is a 50-year-old lawyer living in Boston...,Frank,50,Boston,lawyer,"[golfing, fishing]",True,3,3200000,F
9,"Grace, a 22-year-old student in Austin, is pas...",Grace,22,Austin,student,"[music, volunteering]",False,0,0,G
10,"Katrina, a 28-year-old art expert in NYC.",Katrina,28,NYC,art expert,[],False,0,0,K


In [19]:
extracted_df

,text,name,age,city,occupation,interests,is_married,num_children,net_worth
0,"Alice is 30 years old, lives in New York, and ...",Alice,30,New York,software engineer,"[hiking, reading]",False,1,1200000
1,"Bob, 25, is a teacher in London. He's an avid ...",Bob,25,London,teacher,[cycling],True,2,500000
3,Charlie is a 40-year-old data scientist from C...,Charlie,40,Chicago,data scientist,"[skiing, cooking, photography]",True,1,800000
5,"David, a 35-year-old architect, resides in San...",David,35,San Francisco,architect,[rock climbing],False,0,1500000
6,Emily is a 28-year-old nurse in Seattle. She l...,Emily,28,Seattle,nurse,"[traveling, trying new foods]",False,0,400000
8,Frank is a 50-year-old lawyer living in Boston...,Frank,50,Boston,lawyer,"[golfing, fishing]",True,3,3200000
9,"Grace, a 22-year-old student in Austin, is pas...",Grace,22,Austin,student,"[music, volunteering]",False,0,0
10,"Katrina, a 28-year-old art expert in NYC.",Katrina,28,NYC,art expert,[],False,0,0


In [ ]:
import pandas as pd
from sculptor import Sculptor
from sculptor.helpers.data_sources import CSVDataSource, RedditDataSource
import praw

# Create a Sculptor instance
sculptor = Sculptor(
    extraction_schema={
        "name": {"type": "string", "description": "The entity's name"},
        "summary": {"type": "string", "description": "A summary"}
    },
    template="Entity: {text}"
)

# Get data from a CSV using the helper
csv_source = CSVDataSource("my_data.csv", sep=",")
df_csv = csv_source.get_data()

# Get data from Reddit (you'll need to set up your PRAW Reddit client)
reddit = praw.Reddit(client_id="YOUR_CLIENT_ID", client_secret="YOUR_CLIENT_SECRET", user_agent="YOUR_USER_AGENT")
reddit_source = RedditDataSource(query="artificial intelligence", subreddits=["machinelearning"], reddit_client=reddit)
df_reddit = reddit_source.get_data()

# Apply Sculptor to the DataFrames
df_csv['extracted'] = df_csv.apply(lambda row: sculptor.sculpt(row), axis=1)
df_reddit['extracted'] = df_reddit.apply(lambda row: sculptor.sculpt(row), axis=1)

print(df_csv)
print(df_reddit)

# Test full AI Therapy Pipeline

In [4]:
%load_ext autoreload
%autoreload 2

import os
import toml
import praw
import pandas as pd
from sculptor import Sculptor
from sculptor import SculptorPipeline
from helpers.data_sources import RedditDataSource

secrets = toml.load("secrets.toml")
os.environ["OPENAI_API_KEY"] = secrets["openai"]["api_key"]
os.environ["DEEPINFRA_API_KEY"] = secrets["deepinfra"]["api_key"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
reddit = praw.Reddit(
            client_id=secrets["reddit"]["client_id"],
            client_secret=secrets["reddit"]["client_secret"],
            user_agent=secrets["reddit"]["user_agent"]
        )

reddit_src1 = RedditDataSource(
    reddit_client=reddit,
    query="(AI OR chatbot OR GPT) AND (mental health OR therapy)",
    include_comments=False,
    limit=10
)
reddit_src2 = RedditDataSource(
    reddit_client=reddit,
    query="(AI OR chatbot OR GPT)",
    subreddits=["ADHD", "anxiety"],
    include_comments=False,
    limit=10
)

dfs = [reddit_src1.get_data(), reddit_src2.get_data()]
df = pd.concat(dfs, ignore_index=True).drop_duplicates(subset='id')

In [10]:
pipeline = SculptorPipeline.from_config('examples/ai_therapy.yaml')
results = pipeline.process(df)


Step 1/2


Processing items:   5%|▌         | 1/20 [00:03<01:02,  3.27s/it]

LLM Output: {
    "relevant_sample": true,
    "relevant_sample_explanation": "The user explicitly describes their personal experience of using Chat GPT for mental health support, sharing how it helped them build confidence, reframe negative thoughts, and make progress in a short amount of time. They provide specific details about their interactions with Chat GPT, such as using the voice feature for daily conversations and receiving guidance on overcoming self-esteem issues."
}


Processing items:  10%|█         | 2/20 [00:05<00:52,  2.89s/it]

LLM Output: {"relevant_sample": false, "relevant_sample_explanation": "The post title and text do not indicate a personal experience of using AI for mental health, coaching, or emotional support. Instead, it appears to be a comparative discussion between AI models and mental disabilities, lacking a first-hand account of AI usage for mental health purposes."}


...

LLM Output: {
    "overall_sentiment": 9,
    "benefits": ["emotional regulation", "safe space for venting", "reality check", "practical suggestions"],
    "downsides": null,
    "specific_use_cases": ["venting", "journaling", "emotional regulation"],
    "mentioned_conditions": ["RSD", "ADHD"],
    "provider_status": null,
    "issues_with_human_providers": null,
    "analysis_notes": "User finds ChatGPT helpful for managing emotions and regulating responses during conflicts, using a creative 'hack' of writing fake AITA posts."
}


Processing items:  83%|████████▎ | 5/6 [00:51<00:09,  9.51s/it]

LLM Output: {
    "overall_sentiment": 9,
    "benefits": ["organization", "clarification", "actionable steps", "progress tracking", "memory aid"],
    "downsides": null,
    "specific_use_cases": ["brainstorming", "idea organization", "task planning", "journaling"],
    "mentioned_conditions": ["scatter-brain"],
    "provider_status": null,
    "issues_with_human_providers": null,
    "analysis_notes": "The user utilizes chatGPT to organize their thoughts, turning ideas into actionable steps, and tracking progress. This helps alleviate feelings of lack of accomplishment and provides a memory aid for future reference."
}


Processing items: 100%|██████████| 6/6 [01:03<00:00, 10.59s/it]

LLM Output: {
    "overall_sentiment": 9,
    "benefits": ["improved daily planning", "detailed plan to tackle phobia"],
    "downsides": null,
    "specific_use_cases": ["roleplay", "planning", "phobia management"],
    "mentioned_conditions": ["phobia"],
    "provider_status": null,
    "issues_with_human_providers": null,
    "analysis_notes": "User found AI tool (ChatGPT) helpful for managing phobia and improving daily planning, in conjunction with another tool (goblin.tools)."
}


In [8]:
pipeline.steps[1][0].schema

{'sentiment': {'type': 'integer',
  'description': 'Sentiment towards AI for mental health (1-10, 10 most positive)',
  'items': None,
  'enum': None},
 'benefits': {'type': 'array',
  'description': 'Keywords for benefits: non_judgemental, affordable, accessible, etc',
  'items': 'string',
  'enum': None},
 'downsides': {'type': 'array',
  'description': 'Keywords for downsides: repetitive, shallow, unreliable, etc',
  'items': 'string',
  'enum': None},
 'use_cases': {'type': 'array',
  'description': 'How AI is used: reflection, venting, CBT, journaling, etc',
  'items': 'string',
  'enum': None},
 'conditions': {'type': 'array',
  'description': 'Conditions mentioned: ADHD, depression, anxiety, etc',
  'items': 'string',
  'enum': None},
 'seeing_provider': {'type': 'boolean',
  'description': 'Currently seeing a mental health provider (True or False)',
  'items': None,
  'enum': None},
 'previous_provider': {'type': 'boolean',
  'description': 'Has previously seen a mental health 

In [11]:
results_df = pd.DataFrame(results)
results_df.head()


,id,text,title,context_text,url,subreddit,score,created_utc,is_comment,comment_id,...,relevant_sample_explanation,overall_sentiment,benefits,downsides,specific_use_cases,mentioned_conditions,provider_status,issues_with_human_providers,analysis_notes,conditions
0,1fajq7r_post,I've been desperately trying to figure out wha...,Chat GPT Transforms My Mental Health In 2 Weeks,,https://reddit.com/r/ChatGPT/comments/1fajq7r/...,ChatGPT,775,2024-09-06 16:40:03,False,None,...,The user explicitly describes their personal e...,9,"[increased self-esteem, improved confidence, b...",None,"[CBT, reframing negative thoughts, building co...","[low self-esteem, depression, anxiety]",previously seen provider,[slow progress in therapy],The user credits Chat GPT with helping them id...,NaN
1,1gmmujy_post,Hey!\n\nYou probably heard about people using ...,Using ChatGPT as a tool to improve your mental...,,https://reddit.com/r/DecidingToBeBetter/commen...,DecidingToBeBetter,756,2024-11-08 16:40:52,False,None,...,"The user shares a clear, first-hand experience...",9,"[good advice, new perspectives, feeling unders...",[initial skepticism],"[venting, journaling, problem-solving]","[broken mind, being stuck]",previously seen provider,[didn't work for me],The user is surprised by the effectiveness of ...,NaN
2,1bhxtuf_post,"For me specifically, I am looking for somethin...",What are the best ai assistants or ai tools fo...,,https://reddit.com/r/ADHD/comments/1bhxtuf/wha...,ADHD,24,2024-03-18 18:23:40,False,None,...,The user shares their personal experience and ...,8,"[organization, reminders, prioritization]","[ineffective training, lack of memory]","[executive functioning support, task management]",NaN,not mentioned,not mentioned,The user is seeking an AI assistant to help wi...,[ADHD]
3,1hjkwj8_post,I feel like my RSD plays a huge role in my rel...,ChatGPT Hack,,https://reddit.com/r/ADHD/comments/1hjkwj8/cha...,ADHD,11,2024-12-21 22:31:38,False,None,...,The user explicitly describes a personal exper...,9,"[emotional regulation, safe space for venting,...",None,"[venting, journaling, emotional regulation]","[RSD, ADHD]",None,None,User finds ChatGPT helpful for managing emotio...,NaN
4,1h2iv07_post,This is from a comment I made on some tech pos...,Using “AI” to organize my scatter Brain thoughts,,https://reddit.com/r/ADHD/comments/1h2iv07/usi...,ADHD,0,2024-11-29 10:22:13,False,None,...,The user explicitly describes their personal e...,9,"[organization, clarification, actionable steps...",None,"[brainstorming, idea organization, task planni...",[scatter-brain],None,None,The user utilizes chatGPT to organize their th...,NaN


# Old

In [7]:
CONFIG_TAG = "ai_therapy"
CONFIG_FILE = f"configs/{CONFIG_TAG}.toml"

In [8]:
import toml
import pandas as pd
from openai import OpenAI
from helpers.datasource import RedditDataSource, HackerNewsDataSource

from helpers.processor import Processor, FILTER_PROMPT
import helpers.visualizer as visualizer

secrets = toml.load("secrets.toml")
config = toml.load(CONFIG_FILE)

ANALYSIS_USE_CASE = config["use_case"]
FIELDS = config["fields"]

DEEPINFRA_CONFIG = secrets.get("deepinfra", {})
DEEPINFRA_LLM = OpenAI(
  api_key=DEEPINFRA_CONFIG.get("api_key"),
  base_url=DEEPINFRA_CONFIG.get("base_url"), 
)

OPENAI_CONFIG = secrets.get("openai", {})
OPENAI_LLM = OpenAI(
  api_key=OPENAI_CONFIG.get("api_key"),
  base_url=OPENAI_CONFIG.get("base_url"), 
)

processor = Processor(
    use_case_description=ANALYSIS_USE_CASE,
    filter_prompt=FILTER_PROMPT,
    extraction_schema=FIELDS,
    filter_llm_client=DEEPINFRA_LLM,
    filter_model="meta-llama/Llama-3.3-70B-Instruct-Turbo",  # Cheaper model for filtering
    extract_llm_client=DEEPINFRA_LLM,
    extract_model="meta-llama/Meta-Llama-3.1-405B-Instruct",  # More accurate model for extraction
)

In [3]:
data_sources = []

for ds_conf in config["data_sources"]:
    ds_type = ds_conf.pop("type", None)
    if ds_type == "reddit":
        data_sources.append(RedditDataSource(**ds_conf))
    elif ds_type == "hackernews":
        data_sources.append(HackerNewsDataSource(**ds_conf))
    else:
        print(f"Unknown data source type: {ds_type}")

In [4]:
dfs = [source.get_data() for source in data_sources]
df = pd.concat(dfs, ignore_index=True).drop_duplicates(subset='id')

In [ ]:
# df_filtered = processor.filter_data(df.sample(n=100, random_state=42))  # For testing with a small random sample
df_filtered = processor.filter_data(df)  # 1st level filtering
df_extracted = processor.extract_fields(df_filtered)  # Extraction of structured data
samples = df_extracted[df_extracted['relevant_sample'] == True]  # 2nd level filtering
print(f"Samples: initial={len(df)}, after 1st filter={len(df_filtered[df_filtered['is_relevant']==True])}, final={len(samples)}")

In [ ]:
# # Backup dataframes to output files
# df.to_json(f"out/{CONFIG_TAG}.json", orient='records', date_format='iso')
# df_filtered.to_json(f"out/{CONFIG_TAG}-filtered.json", orient='records', date_format='iso') 
# samples.to_json(f"out/{CONFIG_TAG}-samples.json", orient='records', date_format='iso')

In [ ]:
# Simple plots are created automatically by vizualizer
viz = visualizer.Visualization(samples, FIELDS)
viz.plot_all_fields(show_examples=True)
viz.show_samples(n=3, extra_fields=['sentiment','use_cases'])
viz.plot_by_time('created_utc', "Posts Over Time")
# viz.plot_group_comparison('subreddit', 'sentiment', agg='mean')
# viz.plot_correlation(['sentiment'])